# OANDA Trading Function

This notebook contains a function to place orders with OANDA using specific parameters:
- Credentials
- Instrument
- Price 
- Side
- Stop loss
- Trailing stop loss

In [ ]:
import requests
import json
from datetime import datetime
import pandas as pd
import numpy as np

def create_oanda_order(credentials, instrument, price, side, stop_loss=None, trailing_stop=None):
    """
    Create order(s) with OANDA based on specified parameters.
    
    Parameters:
    -----------
    credentials : dict
        Dictionary containing 'token' and 'account_id' for OANDA API authentication
    instrument : str
        Trading instrument (e.g., 'EUR_USD')
    price : float
        Reference price for order calculation
    side : int
        Order direction: 1 for BUY, -1 for SELL, 0 for both BUY and SELL
    stop_loss : float, optional
        Stop loss in pips
    trailing_stop : float, optional
        Trailing stop in pips
        
    Returns:
    --------
    dict
        Response from OANDA API containing order details
    """
    # Extract credentials
    token = credentials.get('token')
    account_id = credentials.get('account_id')
    
    if not token or not account_id:
        raise ValueError("Credentials must contain 'token' and 'account_id'")
    
    # Define headers for API requests
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    
    # Base URL for OANDA API
    base_url = "https://api-fxpractice.oanda.com/v3"  # Change to fxtrade for live account
    
    # 1. Get account details to check available funds
    account_url = f"{base_url}/accounts/{account_id}"
    
    try:
        response = requests.get(account_url, headers=headers)
        response.raise_for_status()
        account_info = response.json()
        
        # Extract available balance
        available_balance = float(account_info['account']['balance'])
        margin_rate = float(account_info['account']['marginRate'])
        
        print(f"Available Balance: {available_balance}")
        print(f"Margin Rate: {margin_rate}")
        
        # 2. Get current prices for the instrument
        prices_url = f"{base_url}/accounts/{account_id}/pricing"
        params = {'instruments': instrument}
        
        response = requests.get(prices_url, headers=headers, params=params)
        response.raise_for_status()
        price_info = response.json()
        
        # Extract bid and ask prices
        bid_price = float(price_info['prices'][0]['bids'][0]['price'])
        ask_price = float(price_info['prices'][0]['asks'][0]['price'])
        
        print(f"Current Bid: {bid_price}")
        print(f"Current Ask: {ask_price}")
        
        # 3. Calculate order prices (1% above/below current prices)
        buy_price = round(ask_price * 1.01, 5)
        sell_price = round(bid_price * 0.99, 5)
        
        # 4. Calculate maximum units based on available balance and margin
        # This is a simplified calculation - OANDA has more complex margin requirements
        max_units = int((available_balance * 0.95) / (price * margin_rate))
        
        print(f"Maximum Units Available: {max_units}")
        
        # 5. Create orders based on side parameter
        orders_response = []
        
        # Create BUY order if side is 1 or 0
        if side == 1 or side == 0:
            buy_order = {
                "order": {
                    "units": str(max_units),
                    "instrument": instrument,
                    "timeInForce": "GTC",
                    "type": "LIMIT",
                    "positionFill": "DEFAULT",
                    "price": str(buy_price)
                }
            }
            
            # Add stop loss if provided
            if stop_loss:
                buy_order["order"]["stopLossOnFill"] = {
                    "distance": str(stop_loss)
                }
            
            # Add trailing stop if provided
            if trailing_stop:
                buy_order["order"]["trailingStopLossOnFill"] = {
                    "distance": str(trailing_stop)
                }
            
            # Send BUY order request
            order_url = f"{base_url}/accounts/{account_id}/orders"
            response = requests.post(order_url, headers=headers, json=buy_order)
            response.raise_for_status()
            orders_response.append({"BUY": response.json()})
            print("BUY order placed successfully")
        
        # Create SELL order if side is -1 or 0
        if side == -1 or side == 0:
            sell_order = {
                "order": {
                    "units": str(-max_units),  # Negative for SELL
                    "instrument": instrument,
                    "timeInForce": "GTC",
                    "type": "LIMIT",
                    "positionFill": "DEFAULT",
                    "price": str(sell_price)
                }
            }
            
            # Add stop loss if provided
            if stop_loss:
                sell_order["order"]["stopLossOnFill"] = {
                    "distance": str(stop_loss)
                }
            
            # Add trailing stop if provided
            if trailing_stop:
                sell_order["order"]["trailingStopLossOnFill"] = {
                    "distance": str(trailing_stop)
                }
            
            # Send SELL order request
            order_url = f"{base_url}/accounts/{account_id}/orders"
            response = requests.post(order_url, headers=headers, json=sell_order)
            response.raise_for_status()
            orders_response.append({"SELL": response.json()})
            print("SELL order placed successfully")
        
        return orders_response
        
    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")
        if hasattr(e, 'response') and e.response:
            print(f"Response error: {e.response.text}")
        return {"error": str(e)}

## Example Usage

Here's how to use the function with sample credentials:

In [ ]:
# Sample credentials - replace with your actual credentials
sample_credentials = {
    'token': 'your-api-token-here',
    'account_id': 'your-account-id-here'
}

# Example function call (commented out to prevent accidental execution)
'''
result = create_oanda_order(
    credentials=sample_credentials,
    instrument='EUR_USD',
    price=1.1000,  # Reference price
    side=0,        # 0 for both BUY and SELL orders
    stop_loss=0.0050,  # 50 pips stop loss
    trailing_stop=0.0020  # 20 pips trailing stop
)

print(json.dumps(result, indent=2))
'''

## Notes on Implementation

1. **Price Calculation**:
   - BUY orders are placed 1% above the current ask price
   - SELL orders are placed 1% below the current bid price

2. **Order Size**:
   - Uses all available margin (with a 5% safety buffer)
   - Simplified margin calculation - actual implementation may need refinement

3. **Side Parameter**:
   - 1 = BUY only
   - -1 = SELL only
   - 0 = Both BUY and SELL

4. **Stop Loss and Trailing Stop**:
   - Both parameters are optional
   - Values are specified in pips (e.g., 0.0050 = 50 pips)

5. **Error Handling**:
   - Basic error handling included
   - Returns error details if API requests fail